A script to prepare questionnaire data for analysis.

Outputs can be saved at multiple stages according to what you want to analyse, by uncommenting the save as csv lines.

**Change the data directory at the start and the output directory at the end to match those on your PC**

If you have any unicode errors when reading the csv, make sure there is 'r' before the string (pd.read_csv(r'...string goes here...'). 

If this doesn't resolve the issue, repeat the backslashes each time (e.g., C:\\Users\\...), or use forward slashes instead (even if using Windows - e.g., C:/Users/...)

Make sure to have entered an additional column 'Study' in the import file

By Danielle Hewitt, Jan 2023. Last updated, 12th March 2024

In [149]:
import pandas as pd

data = pd.read_csv(r'/Users/dhewitt/Data/Touch/TouchQuestionnaireAll.csv', skiprows=[1,2]) #skipping empty lines from qualtrics
#df = data.loc[:, ~data.columns.isin(['Progress','Duration (in seconds)','Finished','RecordedDate','Gender _6_TEXT','Sex _9_TEXT'])] #removes these cols - old version
df = data.loc[:, ~data.columns.isin(['StartDate', 'EndDate', 'Status', 'IPAddress','Progress','Duration (in seconds)','Finished','RecordedDate','ResponseId','RecipientLastName','RecipientFirstName','RecipientEmail','ExternalReference','LocationLatitude','LocationLongitude','DistributionChannel','UserLanguage','Gender _6_TEXT','Sex _9_TEXT'])] #removes these cols
df.to_csv('/Users/dhewitt/Data/Touch/SA-UK-Collaboration/SA/TouchQuestionnaire_unscored_original.csv', index=False)

#Check if you want by uncommenting
df

,location,STUDY,ID,Age,Gender,Sex,Ethnicity,Q41,Marital_status,religion,...,litpq_want_5_9,litpq_want_6_1,litpq_want_6_2,litpq_want_6_3,litpq_want_6_4,litpq_want_6_5,litpq_want_6_6,litpq_want_6_7,litpq_want_6_8,litpq_want_6_9
0,United Kingdom,2,1,51,Female,Female,§,NaN,Married,Agree,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,United Kingdom,2,2,25,Female,Female,1. English/Welsh/Scottish/Northern Irish/British,NaN,Single,Disagree,...,0.0,5.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0
2,United Kingdom,1,1,21,Female,Female,1. English/Welsh/Scottish/Northern Irish/British,NaN,In a relationship,Strongly disagree,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,United Kingdom,1,2,38,Female,Female,7. White and Asian,NaN,Single,Disagree,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,United Kingdom,1,4,23,Female,Female,1. English/Welsh/Scottish/Northern Irish/British,NaN,In a relationship,Strongly disagree,...,0.0,7.0,3.0,1.0,1.0,1.0,0.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,South Africa,1,DU01,52,Female,Female,"8. Any other Mixed/Multiple ethnic background,...",COLOURED,Divorced,Disagree,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,South Africa,1,BG01,19,Female,Female,14. African,NaN,Single,Strongly agree,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61,South Africa,1,MN01,20,Female,Female,14. African,NaN,Single,Strongly disagree,...,0.0,7.0,5.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0
62,South Africa,1,JT01,20,Female,Female,"4. Any other White background, please describe...",South African,Single,Strongly disagree,...,NaN,10.0,5.0,5.0,3.0,3.0,0.0,0.0,5.0,NaN


In [164]:
recode_dict = {'1. English/Welsh/Scottish/Northern Irish/British' : 1, 
               '4. Any other White background, please describe on next page' : 4,
               '6. White and Black African' : 6,
              '7. White and Asian' : 7,
               '8. Any other Mixed/Multiple ethnic background, please describe on next page' : 8,
               '9. Indian' : 9,
              '10. Pakistani' : 10,
               '12. Chinese' : 12,
               '13. Any other Asian background, please describe on next page' : 13,
               '14. African' : 14,
               '17. Arab' : 17,
               '18. Ethnic group not listed here, please describe on next page' : 18}

# Recode the values in column 'Loc_Cond'
for i in range(df.shape[0]):
    if df.loc[i, 'Ethnicity'] in recode_dict:
        df.loc[i, 'Ethnicity'] = recode_dict[df.loc[i, 'Ethnicity']]
        
recode_dict.clear()

df = df.rename(columns={'Q41': 'EthnicityOther'})

df=df.replace({'Female' : "1", 'United Kingdom' : "UK", 'Yes' : "1", 'No' : '0', 'No ' : '0', 'NO' : '0', 'no' : '0', 'np' : '0'})

# Recode the values in column 'Religion'
recode_dict = {'Strongly agree' : 2,
              'Agree' : 1,
              'Neither agree nor disagree' : 0,
               'Disagree' : -1,
               'Strongly disagree' : -2}

for i in range(df.shape[0]):
    if df.loc[i, 'religion'] in recode_dict:
        df.loc[i, 'religion'] = recode_dict[df.loc[i, 'religion']]
recode_dict.clear()

print(f'Ethnicities, Religion and Gender have been recoded for analysis')
recode_dict.clear()

#Check if you want by uncommenting
#print(df.head())

# Saving everything we have so far after some recoding
#df.to_csv('/Users/dhewitt/Data/Touch/AllQuestionnaireData_recoded.csv', index=False)

# Saving demographic data as a separate dataframe for easier analysis later
demoDf = df.iloc[:,0:22]

Ethnicities, Religion and Gender have been recoded for analysis


Time for summing questionnaire responses, starting with the ECR...

In [165]:
# Getting the file ready for recoding questionnaire responses

AnalysisDf = df.drop(df.columns[6:22], axis=1) #dropping the demographic data for simplicity - add back in at the end

#Next, the recoding. Need to recode each one, iterating over rows of DataFrame, and make a new column which will have the result in

ECR=AnalysisDf.iloc[:,6:42]
ECR=ECR.replace({'Strongly Disagree\n(1)' : "1", 'Disagree\n(2)' : "2", 'Somewhat Disagree\n(3)' : "3", 'Neither Agree Nor Disagree\n(4)' : '4', 'Somewhat Agree\n(5)' : '5', 'Agree\n(6)' : '6', 'Strongly Agree\n(7)' : '7', 'Somewhat Disagree (3)' : "3", 'Neither Agree Nor Disagree' : '4',})

#ECR.columns.get_loc("ECR-R_18.1")

columnnames = [8,10,19,21,25,26,27,28,29,30,32,33,34,35]
reversekeys = {'1' : '7', '2' : '6', '3' : '5', '5' : '3', '6' : '2', '7' : '1'}
ECR.iloc[:,columnnames] = ECR.iloc[:,columnnames].replace(reversekeys)

# Iterate over rows of the DataFrame
for i in ECR.iterrows():
    ECRAnx = i[1][0:18].astype(float).mean(skipna=True)
    ECR.at[i[0], 'ECR_Anx'] = ECRAnx

# Iterate over rows of the DataFrame
for i in ECR.iterrows():
    ECRAtt = i[1][18:36].astype(float).mean(skipna=True)
    ECR.at[i[0], 'ECR_Att'] = ECRAtt

# Iterate over rows of the DataFrame
for i in ECR.iterrows():
    ECRSum = i[1][0:36].astype(float).mean(skipna=True)
    ECR.at[i[0], 'ECR_Sum'] = ECRSum

#And now to add it all together to the AnalysisDf - which we currently aren't using but could be useful for exports without the demographic data
AnalysisDf = pd.concat([AnalysisDf, ECR[['ECR_Anx', 'ECR_Att', 'ECR_Sum']]], axis=1)


Next to sum the TEAQ

In [166]:
#Start by grabbing the data from the other df

TEAQ=AnalysisDf.iloc[:,42:99]

TEAQ = TEAQ.applymap(lambda x: x.strip() if isinstance(x, str) else x) #stripping trailing white space

#Recoding the strings to numeric data
TEAQ=TEAQ.replace({'Disagree strongly (1)' : "1", 'Disagree Strongly (1)' : '1', 'Disagree a little (2)' : "2", 'Neither agree nor disagree (3)' : '3', 'Neither Agree nor Disagree (3)' : '3', 'Agree a little (4)' : '4', 'Agree strongly (5)' : '5', 'Agree a little           (4)' :'4'})

#reverse coding
columnnames = [22,52,8,0,2,27,36,38] #index minus 1 because count starts from 0
reversekeys = {'1' : '5', '2' : '4', '4' : '2', '5' : '1'}
TEAQ.iloc[:,columnnames] = TEAQ.iloc[:,columnnames].replace(reversekeys)

#Now running through each subscale to get a mean value for each one
list=pd.Series([4, 13, 14, 16, 21, 30, 38, 48, 51, 56, 57]).sub(1)
TEAQFFT = TEAQ.iloc[:,list]
TEAQFFT = pd.Series(TEAQFFT.astype(float).mean(axis=1, skipna=True))

list=pd.Series([11, 17, 18, 23, 25, 27, 29, 36, 41, 45, 46, 49, 53, 54]).sub(1)
TEAQCIT = TEAQ.iloc[:,list]
TEAQCIT = pd.Series(TEAQCIT.astype(float).mean(axis=1, skipna=True))

list=pd.Series([5, 6, 9, 15, 22, 32, 33, 35, 42]).sub(1)
TEAQCHT = TEAQ.iloc[:,list]
TEAQCHT = pd.Series(TEAQCHT.astype(float).mean(axis=1, skipna=True))

list=pd.Series([2, 7, 43, 52, 55]).sub(1)
TEAQASC = TEAQ.iloc[:,list]
TEAQASC = pd.Series(TEAQASC.astype(float).mean(axis=1, skipna=True))

list=pd.Series([8, 10, 12, 19, 20, 24, 26, 31, 34, 40, 44, 47, 50]).sub(1)
TEAQAIT = TEAQ.iloc[:,list]
TEAQAIT = pd.Series(TEAQAIT.astype(float).mean(axis=1, skipna=True))

list=pd.Series([1, 3, 28, 37, 39]).sub(1)
TEAQAUT = TEAQ.iloc[:,list]
TEAQAUT = pd.Series(TEAQAUT.astype(float).mean(axis=1, skipna=True))

#TEAQSUM = pd.Series(TEAQ.astype(int).mean(axis=1)) #removed

#Adding to a new dataframe - old, with TEAQSUM
#ALLTEAQ = pd.concat([TEAQFFT, TEAQCIT, TEAQCHT, TEAQASC, TEAQAIT, TEAQAUT, TEAQSUM], axis=1, ignore_index=True)
#ALLTEAQ.columns = ["TEAQ_FFT", "TEAQ_CIT", "TEAQ_CHT", "TEAQ_ASC", "TEAQ_AIT", "TEAQ_AUT", "TEAQ_SUM"]

ALLTEAQ = pd.concat([TEAQFFT, TEAQCIT, TEAQCHT, TEAQASC, TEAQAIT, TEAQAUT], axis=1, ignore_index=True)
ALLTEAQ.columns = ["TEAQ_FFT", "TEAQ_CIT", "TEAQ_CHT", "TEAQ_ASC", "TEAQ_AIT", "TEAQ_AUT"]

#Join to AnalysisDf incase you want to save that later
AnalysisDf = AnalysisDf.join(ALLTEAQ)

#TEAQ.to_csv('/Users/dhewitt/Data/Touch/TEAQ_recoded.csv', index=False)

In [167]:
#Average all touch frequency ('did') and touch wish ('wish'), then divide touch wish by touch
#frequency to calculate the touch deprivation - where higher ratios = greater touch deprivation.
#Change from previous versions where values were summed rather than meaned.

LITPQ=AnalysisDf.iloc[:,99:207]

did = pd.Series(LITPQ.loc[:, LITPQ.columns.str.contains('litpq_did')].mean(axis=1))
want = pd.Series(LITPQ.loc[:, LITPQ.columns.str.contains('litpq_want')].mean(axis=1))
ratio = pd.Series(((want+0.1)/(did+0.1)), name = 'LITPQ_RATIO') #note: added a constant to avoid divide by zero error
AnalysisDf = pd.concat([AnalysisDf, ratio], axis=1)


/var/folders/m9/8l65rzcd5x5903xt73lz9ln00000gn/T/ipykernel_60331/1851803200.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  did = pd.Series(LITPQ.loc[:, LITPQ.columns.str.contains('litpq_did')].mean(axis=1))


In [168]:
AnalysisDf.head()

,location,STUDY,ID,Age,Gender,Sex,ECR-R_1,ECR-R_2,ECR-R_3,ECR-R_4,...,ECR_Anx,ECR_Att,ECR_Sum,TEAQ_FFT,TEAQ_CIT,TEAQ_CHT,TEAQ_ASC,TEAQ_AIT,TEAQ_AUT,LITPQ_RATIO
0,UK,2,1,51,1,1,Disagree\n(2),Disagree\n(2),Strongly Disagree\n(1),Disagree\n(2),...,2.055556,1.666667,1.861111,2.000000,3.500000,3.000000,4.2,4.307692,2.4,0.948359
1,UK,2,2,25,1,1,Somewhat Agree\n(5),Somewhat Agree\n(5),Somewhat Agree\n(5),Somewhat Agree\n(5),...,4.388889,3.444444,3.916667,3.636364,1.928571,4.888889,3.6,3.615385,2.2,1.002124
2,UK,1,1,21,1,1,Disagree\n(2),Disagree\n(2),Disagree\n(2),Disagree\n(2),...,1.666667,1.333333,1.500000,2.363636,4.214286,1.333333,1.4,4.769231,2.0,0.985213
3,UK,1,2,38,1,1,Somewhat Agree\n(5),Somewhat Agree\n(5),Somewhat Agree\n(5),Somewhat Agree\n(5),...,4.222222,2.777778,3.500000,4.000000,3.357143,3.222222,3.2,3.538462,3.6,0.855967
4,UK,1,4,23,1,1,Agree\n(6),Somewhat Agree\n(5),Somewhat Agree\n(5),Somewhat Agree\n(5),...,4.833333,2.111111,3.472222,4.363636,4.357143,5.000000,5.0,5.000000,4.8,2.573904


In [169]:
##THIS WILL BE FOR WORKING OUT LITPQ OVER EACH TOUCH AGENT
#AnalysisDf.columns.get_loc("litpq_want_6_9") #resolve NaNs
#LITPQ=AnalysisDf.iloc[:,99:207]

#ratio = pd.DataFrame()
#for i in range(1, 10):
 #   did = pd.Series(LITPQ.loc[:, LITPQ.columns.str.contains('litpq_did_\d_{i}')].sum(axis=1))
  #  want = pd.Series(LITPQ.loc[:, LITPQ.columns.str.contains('litpq_want_\d_{i}')].sum(axis=1))
   # ratio[f'ratio_{i}'] = want/did #divide by zero error results in error when did is 0 - add a constant?

#LITPQ = pd.concat([LITPQ, ratio], axis=1) #results in NaNs for all data ?? 

In [170]:
#Uncomment to save AnalysisDf to own csv file - optionally by adding the demographic data back in

#AnalysisDf = pd.concat([AnalysisDf, df.iloc[:,6:22]], axis=1) #Demographic data
#AnalysisDf.to_csv('/Users/dhewitt/Data/TouchStudy1/AnalysisQuestionnaireData_recoded.csv', index=False) #Saving all

In [174]:
#Final exports

FinalDf = pd.concat([demoDf, ECR, TEAQ, ALLTEAQ, LITPQ, ratio],axis=1)
FinalDf.to_csv('/Users/dhewitt/Data/Touch/SA-UK-Collaboration/AllQuestionnaireData_SA_UK_recoded.csv', index=False)

#Uncomment to view
FinalDf

,location,STUDY,ID,Age,Gender,Sex,Ethnicity,EthnicityOther,Marital_status,religion,...,litpq_want_6_1,litpq_want_6_2,litpq_want_6_3,litpq_want_6_4,litpq_want_6_5,litpq_want_6_6,litpq_want_6_7,litpq_want_6_8,litpq_want_6_9,LITPQ_RATIO
0,UK,2,1,51,1,1,§,NaN,Married,1,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.948359
1,UK,2,2,25,1,1,1,NaN,Single,-1,...,5.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,1.002124
2,UK,1,1,21,1,1,1,NaN,In a relationship,-2,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.985213
3,UK,1,2,38,1,1,7,NaN,Single,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.855967
4,UK,1,4,23,1,1,1,NaN,In a relationship,-2,...,7.0,3.0,1.0,1.0,1.0,0.0,0.0,3.0,0.0,2.573904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,South Africa,1,DU01,52,1,1,8,COLOURED,Divorced,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.073306
60,South Africa,1,BG01,19,1,1,14,NaN,Single,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.665899
61,South Africa,1,MN01,20,1,1,14,NaN,Single,-2,...,7.0,5.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,1.176367
62,South Africa,1,JT01,20,1,1,4,South African,Single,-2,...,10.0,5.0,5.0,3.0,3.0,0.0,0.0,5.0,NaN,7.550851


In [175]:
#Adding Self Touch Measures Coded from Videos - for CE-UK Collaboration only
data = pd.read_csv('/Users/dhewitt/Data/Touch/CE-UK-Collaboration/UK/selftouch.csv')
study1UK = FinalDf.loc[(FinalDf['STUDY'] == 2) & (FinalDf['location'] == 'UK')]
study1UK['ID'] = pd.to_numeric(study1UK['ID'], errors='coerce').astype('Int32')
study1UK = study1UK.dropna(subset=['ID'])
newdata = pd.merge(study1UK, data, on=["ID", "STUDY"])

/var/folders/m9/8l65rzcd5x5903xt73lz9ln00000gn/T/ipykernel_60331/1614548049.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  study1UK['ID'] = pd.to_numeric(study1UK['ID'], errors='coerce').astype('Int32')


In [176]:
study1 = FinalDf.loc[FinalDf['STUDY'] == 1]
newdata = pd.concat([newdata,study1])
newdata
newdata.to_csv('/Users/dhewitt/Data/Touch/AllQuestionnaireData_withSelfTouch_recoded.csv', index=False)